In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
#import section
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, MinMaxScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [6]:
#read train and test files as pandas dataframes
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")

#first look at the datasets
train.head()
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
#Let's get some info about data in datasets
num_features = [col for col in train.columns if train[col].dtype in ['int64', 'float64']] # numeric features
cat_features = [col for col in train.columns if train[col].dtype == 'object'] # ategorical features
print(num_features)
print(cat_features)
print('______________________________________________________', '\n')

#Let's figure out the amount of missing and unique values in each column of train and test
train_info = pd.DataFrame(index = train.columns)
train_info['Amount_of_miss_val'] = [train[col].isnull().sum() for col in train.columns] # absolute amount of missing values
train_info['Miss_val_percent'] = train_info['Amount_of_miss_val']*100/train.shape[0] # missing values percentage
train_info['Unique_values'] = [train[col].nunique() for col in train.columns] # amount of unique values
print('Train:', train_info)
print('______________________________________________________', '\n')
test_info = pd.DataFrame(index = test.columns)
test_info['Amount_of_miss_val'] = [test[col].isnull().sum() for col in test.columns] # absolute amount of missing values
test_info['Miss_val_percent'] = test_info['Amount_of_miss_val']*100/test.shape[0] # missing values percentage
test_info['Unique_values'] = [test[col].nunique() for col in test.columns] # amount of unique values
print('Test:', test_info)

['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
______________________________________________________ 

Train:              Amount_of_miss_val  Miss_val_percent  Unique_values
PassengerId                   0          0.000000            891
Survived                      0          0.000000              2
Pclass                        0          0.000000              3
Name                          0          0.000000            891
Sex                           0          0.000000              2
Age                         177         19.865320             88
SibSp                         0          0.000000              7
Parch                         0          0.000000              7
Ticket                        0          0.000000            681
Fare                          0          0.000000            248
Cabin                       687         77.104377            147
Embarked                      2       

In [ ]:
# Let's look on featrures with a lot of unique values
print('\n Name:', train['Name'],
      '\n__________________________________________',
      '\nTicket:', train['Ticket'],
      '\n__________________________________________',
      '\nFare:', train['Fare'])

In [ ]:
# As a result of first look on datasets we'll drop 'PassengerId' and 'Ticket' columns 
# because they don't affect at final prediction at all since have a lot of unique values
# From the 'Name' feature we'll try to mine title of each person 
train = train.drop(['PassengerId', 'Ticket'], axis=1)
test_df = test.drop(['PassengerId', 'Ticket'], axis=1)

# We'll round 'Fare' feature
train['Fare'] = round(train['Fare'])

In [ ]:
# Let's provide some EDA on our data

plt.figure(figsize=(10,6))
# At first let's look on our target data - "Survived" column
sb.countplot(x=train.Survived.map({0:'Not-Survived', 1:'Survived'}))
Surv_amount = round(len(train.loc[train['Survived']==1])/len(train)*100, 1)
print("Amount of survived people", Surv_amount, '%') # Almost 62% of people was not survived

# At further let's look at each feature effect
# 'Sex' feature
plt.figure(figsize=(10,6))
sb.countplot(x=train.Sex) # Males are almost as twice as females
plt.figure(figsize=(10,6))
sb.barplot(x=train.Sex, y=train['Survived']) # Females have much more chances to survive

# 'Pclass' feature
sb.countplot(x=train.Pclass) # Most of the peoples travelled by 3-rd class
plt.figure(figsize=(10,6))
sb.barplot(x=train.Pclass, y=train['Survived']) # But the higher the class the higher survival chance rate

# 'Embarked' feature represent the port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)
plt.figure(figsize=(10,6))
sb.countplot(x=train.Embarked) 
plt.figure(figsize=(10,6))
sb.barplot(x=train.Embarked, y=train['Survived']) # People from Cherbourg seems to have a little more chances to Survive

# 'Age' feature
plt.figure(figsize=(10,6))
sb.countplot(x=train['Age']) # Most of the people are 20-40 years old
plt.figure(figsize=(10,6))
sb.barplot(x=train['Age'], y=train['Survived']) # Seems like childs and aged people have more chances to survive

# 'Fare' feature
plt.figure(figsize=(10,6))
sb.countplot(x=train['Fare']) 
plt.figure(figsize=(10,6))
sb.boxplot(x=train['Survived'], y=train['Fare']) # Seems like in average who paid more have more chances to survive

# 'SibSp' feature. Represents № of siblings / spouses
plt.figure(figsize=(10,6))
sb.countplot(x=train['SibSp']) # Most of the people had no siblings/spouses
plt.figure(figsize=(10,6))
sb.barplot(x=train['SibSp'], y=train['Survived']) # People with one or two sib/sp have more chances to survive

# 'Parch' feature. Represents № of parents / childs
plt.figure(figsize=(10,6))
sb.countplot(x=train['Parch']) # Similar to 'SibSp' mos pf the people travelled alone
plt.figure(figsize=(10,6))
sb.barplot(x=train['Parch'], y=train['Survived']) # People with 1,2 or 3 par/ch have more chances to survive

# 'Cabin' feature. We'll skip this feature because of the hig amount of unique values

plt.figure(figsize=(10,6))
#At the final  let's look at correlation heatmap
sb.heatmap(train.corr(), annot=True) # Acording to coorrelation coeff. 'Pclass' have the strongest effect on target, 
                                    #however we don't know for sure if there is a strong linear relation in this ds



In [ ]:
# Let's make some simple Feature engineering
# Let's try to mine title from 'Name' feature
train['Title'] = train.Name.str.split(' ').str[1]
test_df['Title'] = test_df.Name.str.split(' ').str[1]
print('Number of unique titles:', train['Title'].nunique())
print(train['Title'].head())
print('_______________________________________________', '\n', '\n')

# Here we are trying to get Cabin class from 'Cabin' feature
train['Cabin_class'] = train.Cabin.str[0]
test_df['Cabin_class'] = test_df.Cabin.str[0]
print('Number of unique Cabin_class:', train['Cabin_class'].nunique())
print(train['Cabin_class'].head())
print('_______________________________________________', '\n', '\n')

train = train.drop(['Name', 'Cabin'], axis=1) # we don't need 'Name' and 'Cabin' features anymore
test_df = test_df.drop(['Name', 'Cabin'], axis=1)

# And now let's look on our train set
print(train.head()) # we have 10 features (1 target)
print('_______________________________________________', '\n', '\n')
print(train.info()) # 3 features ('Age', 'Embarked', 'Cabin_class') have missing values, 4 features are categorical
print('_______________________________________________', '\n', '\n')

print(test_df.head()) 
print('_______________________________________________', '\n', '\n')
print(test_df.info())
print('_______________________________________________', '\n', '\n')

# Let's assign our train test without target to X, our target - Survived - to y
X = train.drop(['Survived'], axis=1)
y = train['Survived']
print(X.head)
print('_______________________________________________', '\n', '\n')
print(y.head())
# Other data preparation we'll do it by using Scikit Pipeline inside GridSearch

   ** **Now after simple feature engineering our dataset still contain missing values and categorical values, thus, we need to preprocess dataset before model building. All of that stuff, like handling missing values, encoding categorical features we will do using Pipeline inside of GridSearch. So what we are going to do is to construct two separate preprocessing Pippelines for numerical and categorical columns, then we'll combine these Pipelines into one preprocessor by ColumnTransformer. Then we'll create another Pipeline that will contain our preprocessor and some model. Thus our final Pipeline will be able to prepare our dataset and model it. 
    However, we want to find the best prerossesing approaches and best model for this dataset, so we will pass our final Pipeline to GridSearch and provide it with a bunch of different preprocessing steps and model hyperparameters. As a result GridSearch will find the best preprocessing parameters and best model hyperparameters. In order to test different models we are going to create several GridSearch estimators with different models and after that we will find can find the best model with optimized hyperparameters.****

In [ ]:
# At first let's define pipeline_constructor function that constructs preprocessor for handling our date before modeling
def pipeline_constructor():
    """
    Construct Pipelines for numerical and categorical columns.
    Create self.preprocessor and self.preprocessor_dict for final Pipeline and further GridSearch
    """

    # At first we'll find columns with numerical and categorical values
    num_features = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
    cat_features = [col for col in X.columns if X[col].dtype == 'object']

    # Pipeline for numerical features that contain imputing, scaling and normalization operations
    num_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer()),
        ('scaler', 'passthrough'),
        ('norm', 'passthrough')
    ])

    # Pipeline for categorical features that contain imputing and one-hot encoding operations
    cat_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Combining of num and cat Pipelines into one preprocessor step using ColumnTransformer.
    # This preprocessor will be used in final Pipeline and further in GridSearch
    preprocessor = ColumnTransformer(transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ])

    # Creating of dictionary with preprocessing parameters. This dict will be used in GridSearch
    preprocessor_dict = dict(preprocessor__num__imputer__strategy=['mean', 'median'],
                             preprocessor__num__scaler=[StandardScaler(), RobustScaler(),
                                                        RobustScaler(with_centering=False), MinMaxScaler()],
                                preprocessor__num__norm=[PowerTransformer()])
    return preprocessor, preprocessor_dict

In [ ]:
# Now we'll define three function that will be able to provide GridSearch with different models.
# GridSearch also will use above defined preprocessor: 

preprocessor, preprocessor_dict =  pipeline_constructor()

def RFC_grid_search(grid_params={}):
    """
    Method create classifier using preprocessor and RandomForestClassifier and fit this Pipeline to X, y

    """
    RFC_classifier = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier())
    ])
    RFC_dict = dict(classifier__n_estimators=[100, 500, 900],  # classifier hyperparameters dict
                    classifier__min_samples_leaf=[1, 2, 4])

    grid_params.update(preprocessor_dict)
    grid_params.update(RFC_dict) # dict used in GridSearch as param_grid

    grid_search = GridSearchCV(RFC_classifier, param_grid=grid_params, scoring='roc_auc', n_jobs=-1)
    grid_search.fit(X, y)

    return grid_search.best_score_, grid_search.best_estimator_, grid_search.best_params_
    
def SVC_grid_search(grid_params={}):
    """
    Method create classifier using preprocessor and SVC and fit this Pipeline to X, y

    """
    SVC_classifier = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', SVC(probability=True))
    ])
    SVC_dict = dict(classifier__kernel=['rbf', 'linear', 'sigmoid'],  # classifier hyperparameters dict
                    classifier__C=[0.001, 0.01, 0.1, 1])
    grid_params.update(preprocessor_dict)
    grid_params.update(SVC_dict)

    grid_search = GridSearchCV(SVC_classifier, param_grid=grid_params, scoring='roc_auc', n_jobs=-1)
    grid_search.fit(X, y)
    return grid_search.best_score_, grid_search.best_estimator_, grid_search.best_params_

def SGD_grid_search(grid_params={}):
    """
    Method create classifier using preprocessor and SGD and fit this Pipeline to X, y

    """
    SGD_classifier = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', SGDClassifier(early_stopping=True))
    ])
    SGD_dict = dict(classifier__alpha=[0.001, 0.01, 0.1, 1])  # classifier hyperparameters dict
    grid_params.update(preprocessor_dict)
    grid_params.update(SGD_dict)

    grid_search = GridSearchCV(SGD_classifier, param_grid=grid_params, scoring='roc_auc', n_jobs=-1)
    grid_search.fit(X, y)
    return grid_search.best_score_, grid_search.best_estimator_, grid_search.best_params_

# Finaly we'll create function that will be choose the best GridSearh estimator from the above defined
def grid_search_estimator(estimators_dict={}):
    """
    Method choose best Pipeline basing on best_score

    """
    rfc_score, rfc_best_estimator, rfc_best_params_ = RFC_grid_search()
    svc_score, svc_best_estimator, svc_best_params_ = SVC_grid_search()
    sgd_score, sgd_best_estimator, sgd_best_params_ = SGD_grid_search()

    estimators_dict['RFC'] = [rfc_score, rfc_best_estimator]
    estimators_dict['SV'] = [svc_score, svc_best_estimator]
    estimators_dict['SGD'] = [sgd_score, sgd_best_estimator]
    score_list = [i[0] for i in list(estimators_dict.values())]

    f = 0
    max_idx = 0
    for j in range(len(score_list)):
        if score_list[j] > f:
            f = score_list[j]
            max_idx = j

    # max_idx = list.index(max(estimators_list))
    print('best_estimator:', list(estimators_dict.keys())[max_idx],
            '\nbest_score:', list(estimators_dict.values())[max_idx][0])
    if max_idx == 0:
        return rfc_best_estimator, rfc_score
    elif max_idx == 1:
        return svc_best_estimator, svc_score
    else:
        return sgd_best_estimator, sgd_score

In [ ]:
# Let's find teh best GridSearh estimator
best_estimator, best_score = grid_search_estimator()
fitted_model = best_estimator.fit(X, y)

In [ ]:
# Using the best GridSearh estimator we will predict our test dataset
test_pred = fitted_model.predict(test_df)
print(test_pred)

In [ ]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': test_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")
print(output)